## Hydra-zen

### Generate Hydra compatible cfg

In [ ]:
import omegaconf
import pyrootutils

# root = pyrootutils.setup_root("/root/workspace/EP2", pythonpath=True)
cfg = omegaconf.OmegaConf.load("../configs/callbacks/eqprop_track.yaml")

In [ ]:
omegaconf.OmegaConf.register_new_resolver(
    "double", lambda condition, false_value: 2 * false_value if condition else false_value
)

In [ ]:
cfg.net.input_size

In [ ]:
print(omegaconf.OmegaConf.to_yaml(cfg))

In [ ]:
from hydra.utils import instantiate

instantiate(cfg)

In [ ]:
import hydra_zen

hydra_zen.instantiate(cfg)

### Make Hierachial configs

see https://github.com/mit-ll-responsible-ai/hydra-zen-examples

In [ ]:
import math

import pytorch_lightning as pl
import torch as tr
from hydra_zen import builds, instantiate, make_config, make_custom_builds_fn
from torch.optim import Adam
from torch.utils.data import DataLoader
from zen_model import UniversalFuncModule, single_layer_nn

pbuilds = make_custom_builds_fn(zen_partial=True, populate_full_signature=True)

OptimConf = pbuilds(Adam)

LoaderConf = pbuilds(DataLoader, batch_size=25, shuffle=True, drop_last=True)

ModelConf = builds(single_layer_nn, num_neurons=10)

# configure our lightning module
LitConf = pbuilds(
    UniversalFuncModule,
    model=ModelConf,
    target_fn=tr.cos,
    training_domain=builds(tr.linspace, start=-2 * math.pi, end=2 * math.pi, steps=1000),
)

TrainerConf = builds(pl.Trainer, max_epochs=100)

ExperimentConfig = make_config(
    optim=OptimConf,
    dataloader=LoaderConf,
    lit_module=LitConf,
    trainer=TrainerConf,
    seed=1,
)


def task_function(cfg):
    # cfg: ExperimentConfig
    pl.seed_everything(cfg.seed)

    obj = instantiate(cfg)

    # finish instantiating the lightning module, data-loader, and optimizer
    lit_module = obj.lit_module(dataloader=obj.dataloader, optim=obj.optim)

    # train the model
    obj.trainer.fit(lit_module)

    # evaluate the model over the domain to assess the fit
    data = lit_module.training_domain
    final_eval = lit_module.forward(data.reshape(-1, 1))
    final_eval = final_eval.detach().cpu().numpy().ravel()

    # return the final evaluation of our model:
    # a shape-(N,) numpy-array
    return final_eval

# Hydra

In [ ]:
import hydra

@hydra.main(config_path="configs", config_name="model/mnist")

In [ ]:
import pytest
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

from src.models.components.eqprop_backbone import AnalogEP2
from src.models.eqprop_module import EqPropLitModule


@pytest.fixture
def eqprop_model():
    optimizer = optim.Adam(lr=0.001, weight_decay=0.0)
    scheduler = lr_scheduler.ReduceLROnPlateau(mode="min", factor=0.1, patience=10)
    net = AnalogEP2(input_size=2, lin1_size=128, output_size=10)
    model = EqPropLitModule(net, optimizer, scheduler)
    return model


def test_optimizer(eqprop_model):
    assert isinstance(eqprop_model.optimizer, optim.Adam)
    assert eqprop_model.optimizer.defaults["lr"] == 0.001
    assert eqprop_model.optimizer.defaults["weight_decay"] == 0.0


def test_scheduler(eqprop_model):
    assert isinstance(eqprop_model.scheduler, lr_scheduler.ReduceLROnPlateau)
    assert eqprop_model.scheduler.mode == "min"
    assert eqprop_model.scheduler.factor == 0.1
    assert eqprop_model.scheduler.patience == 10


def test_net(eqprop_model):
    assert isinstance(eqprop_model.net, AnalogEP2)
    assert eqprop_model.net.input_size == 2
    assert eqprop_model.net.lin1_size == 128
    assert eqprop_model.net.output_size == 10


def test_double_input(eqprop_model):
    assert eqprop_model.net.input_size == 4


def test_double_output(eqprop_model):
    assert eqprop_model.net.output_size == 20


def test_positive_w(eqprop_model):
    assert (eqprop_model.net.w > 0).all()


def test_bias(eqprop_model):
    assert eqprop_model.net.b is None


def test_clip_weights(eqprop_model):
    assert (eqprop_model.net.w.abs() <= 1).all()

In [ ]:
# instantiate the config